In [1]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pymaster as nmt

In [ ]:
nreal = 10
nside = 128
lmax = 3*nside-1
npix = hp.nside2npix(nside)
b = nmt.NmtBin.from_nside_linear(nside, 32, is_Dell=True)
ell = b.get_effective_ells()

total = np.loadtxt('totcls.txt')
ell_theory, TT, EE, BB, TE = np.transpose(total[0:lmax])
mask = hp.read_map('./out_maps/0/lcdm_telescope_all_time_all_invnpp.fits', field=[0], verbose=False, dtype=np.float64)
mask[np.where(mask>0)] = 1
mask_apo = nmt.mask_apodization(mask, 0.3, apotype='C1')

In [ ]:
hp.mollview(mask)

In [ ]:
cl_in = []
cl_out = []
cl_pure = []

for i in range(nreal):
    #INPUT MAPS
    f_in = nmt.NmtField(mask=np.ones(npix), maps=hp.read_map(f'./input_maps/map_{i}.fits', field=[1,2], verbose=False, dtype=np.float64))
    cl_in.append(nmt.compute_full_master(f_in, f_in, b))

    #OUTPUT MAPS
    f_out = nmt.NmtField(mask=mask_apo, maps=hp.read_map(f'./out_maps/{i}/lcdm_telescope_all_time_all_binned.fits', field=[1,2], verbose=False, dtype=np.float64), masked_on_input=False)
    cl_out.append(nmt.compute_full_master(f_out, f_out, b))
    
    #PURIFICATION
    f_pure = nmt.NmtField(mask=mask, maps=hp.read_map(f'./out_maps/{i}/lcdm_telescope_all_time_all_binned.fits', field=[1,2], verbose=False, dtype=np.float64), masked_on_input=False, purify_b=True)
    cl_pure.append(nmt.compute_full_master(f_pure, f_pure, b))

In [ ]:
# plt.plot(ell_theory, EE, label='Theory EE')
# plt.plot(ell_theory, BB, label='Theory BB (r=0.01)')

plt.errorbar(ell, np.mean(cl_in, axis=0)[0], yerr=np.std(cl_in, axis=0)[0], label='input EE, full sky')
plt.errorbar(ell, np.mean(cl_in, axis=0)[3], yerr=np.std(cl_in, axis=0)[3], label='input BB, full sky')
plt.errorbar(ell, np.mean(cl_out, axis=0)[0], yerr=np.std(cl_out, axis=0)[0], label='output EE')
plt.errorbar(ell, np.mean(cl_out, axis=0)[3], yerr=np.std(cl_out, axis=0)[3], label='output BB')
plt.errorbar(ell, np.mean(cl_pure, axis=0)[3], yerr=np.std(cl_pure, axis=0)[3], label='output BB, purified')

plt.yscale('log')
plt.xscale('log')

plt.legend()
plt.xlabel('$\ell$')
plt.ylabel('$D_\ell$')